In [1]:
import numpy as np
from numpy import savetxt
from numpy import loadtxt
import fitsio
import healpy as hp
import matplotlib.pyplot as plt
import Config
from astropy.table import Table

In [2]:
numBins = 100
# numBinsTrained = 100
conditions = Config.conditions
file = '/hdfs/bechtol/balrog/y3/y3a2_survey_conditions_maps/Kyle_Stuff/training/'

In [3]:
validPix = fitsio.read(file + 'Valid_4096_Pixels.fits')['PIXEL']

In [4]:
stds = np.loadtxt(file + 'Blue_Standard_Deviations.csv', delimiter = ',')
mean = np.loadtxt(file + 'Blue_Means.csv', delimiter = ',')
inds = np.loadtxt(file + 'Blue_Indices.csv', delimiter = ',').astype(int)

In [5]:
strFitParams = loadtxt(file + 'Blue_Fit_Parameters.csv', delimiter = '\t', dtype = object)
fitParams = []
for i in range(len(strFitParams)):
    
    strFitParams[i] = strFitParams[i][1:len(strFitParams[i]) - 1]
    indParam = []
    
    for j in range(len(strFitParams[i].split(', '))):
        
        indParam.append(float(strFitParams[i].split(', ')[j]))
        
    indParam = np.array(indParam)
    fitParams.append(indParam)
    
fitParams = np.array(fitParams, dtype = object)

In [6]:
strFeatVec = loadtxt(file + 'Blue_Feature_Vectors.csv', delimiter = '\t', dtype = object)
featVec = []
for i in range(len(strFeatVec)):
    
    indVec = []
    
    for j in range(len(strFeatVec[i])):
        
        indVec.append(float(strFeatVec[i][j]))
        
    indVec = np.array(indVec)
    featVec.append(indVec)
    
featVec = np.array(featVec, dtype = object)

In [7]:
binLims = [0]

for i in range(numBins):
    binLims.append(int((len(validPix) - binLims[-1]) / (numBins - i)) + (binLims[-1]))

In [8]:
corrMap = []

for i in range(len(binLims) - 1):
    
    condMaps = []
    # This loops over every condition file except for stellar density which has a different format
    for cond in conditions:
        condMaps.append(fitsio.read(file + cond + '.fits', rows = np.arange(binLims[i], binLims[i + 1]))['SIGNAL']) # This reads in the data
    condMaps = np.array(condMaps, dtype = object)    
    
    stanMaps = []
    for j in range(len(condMaps)):
        stanMaps.append((condMaps[j] - mean[j]) / stds[j])
    stanMaps = np.array(stanMaps, dtype = object)
    
    redMaps = np.matmul(featVec, stanMaps)
    
    sectCorr = np.ones(len(stanMaps[0]))
    
    for j in range(len(inds)):
    
        corr = np.full(len(sectCorr), 0.0)

        for k in range(len(fitParams[j])):
            corr = corr + (fitParams[j][k] * (redMaps[inds[j]]**(len(fitParams[j]) - 1 - k)))

        corr = 1 / corr
        
        sectCorr = sectCorr * corr
        
    corrMap.extend(sectCorr)
    
corrMap = np.array(corrMap)

In [9]:
my_table = Table()
my_table['SIGNAL'] = corrMap
my_table.write(file + 'Blue_Corrections_Map.fits', overwrite = True)